In [2]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns

In [4]:
bikes = pd.read_csv("/Users/pablosoriano/Documents/Data Science/Code Academy/projects/project_3_ capitalbikeshare/CBS_2021-2023_Full.csv", low_memory=False)

# Data Wrangling

In [5]:
bikes.head(10)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member
5,0FD3B4B2152425C3,electric_bike,2021-01-01 00:18:16,2021-01-01 00:29:05,NaN,NaN,Georgia Ave & Kennedy St NW,31415.0,38.930000,-77.030000,38.956738,-77.027940,casual
6,0535306B61EF92F9,classic_bike,2021-01-01 00:19:06,2021-01-01 00:36:28,Bladensburg Rd & Benning Rd NE,31617.0,Good Hope Rd & MLK Ave SE,31802.0,38.900413,-76.982872,38.867373,-76.988039,casual
7,78894BDABD9F21DD,electric_bike,2021-01-01 00:20:40,2021-01-01 00:35:36,Columbia Rd & Georgia Ave NW,31115.0,NaN,NaN,38.928081,-77.023768,38.900000,-77.020000,casual
8,CB36F43FA77E4A35,docked_bike,2021-01-01 00:21:19,2021-01-01 00:33:31,15th & F St NE,31632.0,1st & M St SE,31650.0,38.897195,-76.983575,38.876893,-77.006072,casual
9,874D92427D41BC30,classic_bike,2021-01-01 00:21:39,2021-01-01 00:31:07,14th & Otis Pl NW,31131.0,14th & R St NW,31202.0,38.934405,-77.032687,38.913046,-77.032008,member


In [9]:
bikes.shape

(10693997, 13)

In [ ]:
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693997 entries, 0 to 10693996
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 1.0+ GB


In [15]:
bikes["started_at"]=pd.to_datetime(bikes["started_at"])
bikes["ended_at"]=pd.to_datetime(bikes["ended_at"])
bikes.info()
bikes.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10693997 entries, 0 to 10693996
Data columns (total 13 columns):
 #   Column              Dtype         
---  ------              -----         
 0   ride_id             object        
 1   rideable_type       object        
 2   started_at          datetime64[ns]
 3   ended_at            datetime64[ns]
 4   start_station_name  object        
 5   start_station_id    object        
 6   end_station_name    object        
 7   end_station_id      object        
 8   start_lat           float64       
 9   start_lng           float64       
 10  end_lat             float64       
 11  end_lng             float64       
 12  member_casual       object        
dtypes: datetime64[ns](2), float64(4), object(7)
memory usage: 1.0+ GB


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member


## handling missing / incomplete data

In [22]:
# missing values
bikes.isna().any()
bikes.isna().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    742776
start_station_id      742776
end_station_name      811282
end_station_id        811282
start_lat                  2
start_lng                  2
end_lat                16765
end_lng                16765
member_casual              0
dtype: int64

maybe input missing starting and end station with the details from lat/lng? 
can also check the id of the stations to check for missing names and viceversea / nope, if they have the name, they have the id

In [28]:
bikes[bikes["start_station_name"]=="10th & U St NW"].sort_values(by="start_lat")

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
10506149,6A6CAD5517A4FA1D,electric_bike,2023-12-08 22:20:52,2023-12-08 22:24:56,10th & U St NW,31111.0,14th St & Rhode Island Ave NW,31203.0,38.916716,-77.026061,38.908600,-77.032300,casual
2211711,DD7B9A7B33FA967D,electric_bike,2021-10-21 18:09:31,2021-10-21 18:21:59,10th & U St NW,31111.0,North Capitol & R St NE,31527.0,38.916757,-77.025588,38.912556,-77.008685,casual
10154847,D9A24626EDCD4707,electric_bike,2023-11-07 16:22:26,2023-11-07 16:30:39,10th & U St NW,31111.0,14th & Newton St NW,31649.0,38.916867,-77.025362,38.931991,-77.032956,member
7057282,C9B83877C9D2C03A,electric_bike,2023-04-11 05:50:16,2023-04-11 05:54:03,10th & U St NW,31111.0,NaN,NaN,38.916912,-77.026125,38.920000,-77.040000,member
9634158,FDF6AF3682C023FE,electric_bike,2023-10-04 22:03:49,2023-10-04 22:07:30,10th & U St NW,31111.0,NaN,NaN,38.916914,-77.025880,38.920000,-77.030000,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9973461,0BE7488B0E9C1265,electric_bike,2023-10-26 21:42:39,2023-10-26 21:46:03,10th & U St NW,31111.0,NaN,NaN,38.917503,-77.025957,38.920000,-77.030000,member
9240437,4C163E6DE5F19ADB,electric_bike,2023-09-09 00:46:37,2023-09-09 01:02:57,10th & U St NW,31111.0,1st & L St NW,31677.0,38.917532,-77.025985,38.903819,-77.011987,casual
4298303,4823483E3200BAA3,electric_bike,2022-06-30 19:36:10,2022-06-30 19:52:56,10th & U St NW,31111.0,14th & Upshur St NW,31406.0,38.919537,-77.025082,38.942475,-77.032678,casual
10465730,7B584527BD6DBCCE,electric_bike,2023-12-05 15:42:34,2023-12-05 16:13:23,10th & U St NW,31111.0,NaN,NaN,38.951712,-77.002611,38.920000,-77.030000,member


maybe dont need to check for lat and len, and only len is enough -  NOPE
Check if it's possible that there are more than one station in a given lat - yes, it's possible
not possible to do location by location, but can arrange it by area
check max and min values, that's the range

determine the latitude/long coordinates for each station
do a for loop creating a range of max and min, latitude and longitud, for each station. 
Use this range to input missing values.

In [44]:
# transform coordinates into float
bikes2= bikes.copy()

# bikes2.astype({"start_lat": float})
bikes2.astype({"start_lat": float, "start_lng": float, "end_lat":float, "end_lng":float})

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646.0,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015.0,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111.0,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214.0,14th & Belmont St NW,31119.0,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926.0,Wilson Blvd. & N. Vermont St.,31926.0,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651.0,14th & Newton St NW,31649.0,38.931322,-77.028247,38.931991,-77.032956,member
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10693992,1E69FA72F720F2E7,classic_bike,2023-12-31 23:58:06,2024-01-01 00:12:29,King St & Peyton St,31955.0,Market Square / King St & Royal St,31042.0,38.806002,-77.055911,38.804718,-77.043363,member
10693993,BA9E9FDBFAFDE618,electric_bike,2023-12-31 23:58:27,2024-01-01 00:02:28,Eastern Market Metro / Pennsylvania Ave & 8th ...,31613.0,Lincoln Park / 13th & East Capitol St NE,31619.0,38.884185,-76.995396,38.890461,-76.988355,member
10693994,F3B6C6BFB1F738D0,classic_bike,2023-12-31 23:59:27,2024-01-01 02:09:11,22nd & P ST NW,31285.0,23rd & E St NW,31260.0,38.909394,-77.048728,38.896104,-77.049882,casual
10693995,D2DB18DB58BC83D2,classic_bike,2023-12-31 23:59:30,2024-01-01 02:02:07,22nd & P ST NW,31285.0,22nd & P ST NW,31285.0,38.909394,-77.048728,38.909394,-77.048728,casual


In [61]:

bikes2["start_station_id"]=bikes2["start_station_id"].str.replace(".0","")
#FIXME - cannot convert float NaN to integer. need to deal with the NaN first
bikes2["start_station_id"].astype("int", errors="ignore")

bikes2["end_station_id"]=bikes2["end_station_id"].str.replace(".0","")
bikes2["end_station_id"].astype("int", errors="ignore")

bikes2.head()




,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,5CB9DFCECF79AF84,classic_bike,2021-01-01 00:08:33,2021-01-01 00:33:53,Maine Ave & 9th St SW,31646,Rosslyn Metro / Wilson Blvd & Ft Myer Dr,31015,38.880440,-77.025236,38.894600,-77.072305,member
1,629E059504606547,electric_bike,2021-01-01 00:13:43,2021-01-01 00:29:34,10th & U St NW,31111,NaN,NaN,38.917193,-77.025894,38.960000,-77.020000,casual
2,E74069873161EE33,electric_bike,2021-01-01 00:14:32,2021-01-01 00:28:45,17th & Corcoran St NW,31214,14th & Belmont St NW,31119,38.912138,-77.038568,38.920870,-77.031691,member
3,91F95E512CABC46A,classic_bike,2021-01-01 00:15:45,2021-01-01 00:21:20,Wilson Blvd. & N. Vermont St.,31926,Wilson Blvd. & N. Vermont St.,31926,38.879477,-77.114563,38.879477,-77.114563,member
4,DA46A05139C0EA2F,classic_bike,2021-01-01 00:17:46,2021-01-01 00:21:00,11th & Park Rd NW,31651,14th & Newton St NW,31649,38.931322,-77.028247,38.931991,-77.032956,member


## unique values

In [ ]:
# unique values before formatting
# Filter the DataFrame for object columns
non_numeric = bikes.select_dtypes("object")

# Loop through columns
for x in non_numeric.columns:
  
  # Print the number of unique values
  print(f"Number of unique values in {x} column: ", non_numeric[x].nunique())

# Filter the DataFrame for object columns
non_numeric = bikes2.select_dtypes("object")

# Loop through columns
for x in non_numeric.columns:
  
  # Print the number of unique values
  print(f"Number of unique values in {x} column: ", non_numeric[x].nunique())

  ##NOTE - removing the '.0" at the end of station id reduced the amount of unique values by 600


## Dealing with missing values

In [ ]:
# Determine a range of latitude and longitud (max and min) for each station
coordinate_columns = bikes2[["start_lat","start_lng","end_lat","end_lng"]]

# bikes2[bikes2["start_station_name"]=="10th & U St NW"].nlargest(1, "start_lat")
print("max_latitude: ", bikes2[bikes2["start_station_name"]=="10th & U St NW"].nlargest(1,"start_lat")["start_lat"])
print("min_latitude: ",bikes2[bikes2["start_station_name"]=="10th & U St NW"].nsmallest(1,"start_lat")["start_lat"])

# for loop to determine max and min lat of each unique stations(id)
unique_stations = nunique()
# for station_id in bikes2["start_station_id"]:


max_latitude:  10395364    38.983053
Name: start_lat, dtype: float64
min_latitude:  10506149    38.916716
Name: start_lat, dtype: float64


In [ ]:
# for loop to determine max and min lat of each unique stations(id)
unique_stations = bikes2["start_station_id"].unique()
unique_stations
max_lat={}
min_lat={}
for station_id in unique_stations:
    bikes2[bikes2["start_station_id"]==station_id].nlargest(1,"start_lat")


how is it possible that there are more unique values in the id columns than in the name columns? they should be the same. one possible explanation is that one station_name has multiple station_id's